# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [ ]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import torch
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np

from workspace_utils import active_session

from pathlib import Path

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [ ]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [ ]:
# Knobs and switches
dataloader_batch_size = 64
qty_output_units = 102
learn_rate = 0.001
epochs = 2

# ImageNet normalization values
mean_norm = [0.485, 0.456, 0.406]
std_norm = [0.229, 0.224, 0.225]

normalize_xform = transforms.Normalize(mean_norm, std_norm)

resize_crop_only_xform = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize_xform
])

In [ ]:
# TODO: Define your transforms for the training, validation, and testing sets

# For training set, apply random transformations like cropping, scaling, rotating
# For validation and test sets, apply no transformations except for resize and crop.

data_transforms = {
    'training' : transforms.Compose([
        transforms.RandomRotation(30),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize_xform]),
    'validation' : resize_crop_only_xform,
    'test' : resize_crop_only_xform
}

# TODO: Load the datasets with ImageFolder
image_datasets = {
    'training' : datasets.ImageFolder(
        train_dir, transform = data_transforms['training']),
    'validation' : datasets.ImageFolder(
        valid_dir, transform = data_transforms['validation']),
    'test' : datasets.ImageFolder(
        test_dir, transform = data_transforms['test'])
}

# TODO: Using the image datasets and the transforms, define the dataloaders
dataloaders = {
    'training' : DataLoader(
        image_datasets['training'], 
        batch_size=dataloader_batch_size, shuffle=True),
    'validation' : DataLoader(
        image_datasets['validation'], 
        batch_size=dataloader_batch_size, shuffle=True),
    'test' : DataLoader(
        image_datasets['test'], 
        batch_size=dataloader_batch_size, shuffle=True)
}

In [ ]:
# View sample of transformed images
def imshow_preview(image, mean_norm, std_norm, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean, std = mean_norm, std_norm
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

# Select training, validation, or test set here by commenting/commenting out
loader_select = 'training'
#loader_select = 'validation'
#loader_select = 'test'

data_iter = iter(dataloaders[loader_select])

images, labels = next(data_iter)

# Edit number of images to show here
subplot_rows, subplot_cols = 2, 4

fig, axes = plt.subplots(nrows=subplot_rows, ncols=subplot_cols, 
                         figsize=(10,5))

for row in range(subplot_rows):
    for col in range(subplot_cols):
        ax = axes[row][col]
        imshow_preview(images[row*subplot_cols+col], mean_norm, std_norm, ax=ax)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [ ]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [ ]:
# TODO: Build and train your network

In [ ]:
# BUILD
model = models.vgg19(pretrained=True)

# Freeze features
for param in model.parameters():
    param.requires_grad = False
    
from collections import OrderedDict
classifier_struct = OrderedDict([
    ('fc1', nn.Linear(25088, 4096)),
    ('relu1', nn.ReLU()),
    ('dropout1', nn.Dropout(0.2)),
    ('fc2', nn.Linear(4096, 1024)),
    ('relu2', nn.ReLU()),
    ('dropout2', nn.Dropout(0.2)),
    ('fc3', nn.Linear(1024, qty_output_units)),
    ('output', nn.LogSoftmax(dim=1))
    ])
classifier = nn.Sequential(classifier_struct)

model.classifier = classifier

criterion = nn.NLLLoss()

optimizer = optim.Adam(model.classifier.parameters(), lr=learn_rate)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# TRAIN
steps = 0
running_loss = 0
print_every = 50

print(f"Beginning training with {len(dataloaders['training'])}\
 images and {epochs} epochs\n")

with active_session():
    model.train()
    for epoch in range(epochs):
        for inputs, labels in dataloaders['training']:
            steps += 1
            # Move input and label tensors to the default device
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            logps = model.forward(inputs)
            loss = criterion(logps, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            step_of_epoch = steps % len(dataloaders['training'])
            if steps % print_every == 0 or step_of_epoch == 0:
                test_loss = 0
                accuracy_sum = 0
                qty_processed_inputs = 0
                model.eval()
                with torch.no_grad():
                    # One iteration per batch
                    for inputs, labels in dataloaders['validation']:
                        inputs, labels = inputs.to(device), labels.to(device)
                        logps = model.forward(inputs)
                        batch_loss = criterion(logps, labels)

                        test_loss += batch_loss.item()

                        # Calculate accuracy
                        ps = torch.exp(logps)
                        top_p, top_class = ps.topk(1, dim=1)
                        equals = top_class == labels.view(*top_class.shape)
                        qty_processed_inputs += len(equals)
                        accuracy_sum += torch.sum(equals.type(torch.FloatTensor)).item()
                    
                print(f"Epoch {epoch+1}/{epochs}, after "
                      f"training round {len(dataloaders['training']) if step_of_epoch == 0 else step_of_epoch}/{len(dataloaders['training'])}.. "
                      f"Train loss: {running_loss/print_every:.3f}.. "
                      f"Validation loss: {test_loss/len(dataloaders['validation']):.3f}.. "
                      f"Validation accuracy: {accuracy_sum/qty_processed_inputs:.3f}")
                running_loss = 0
                model.train()

print('Training complete!')

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# TODO: Do validation on the test set
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# TEST
print(f"Beginning testing with {len(dataloaders['test'])} images\n")

with active_session():
    test_loss = 0
    accuracy_sum = 0
    qty_processed_inputs = 0
    model.eval()
    with torch.no_grad():
        # One iteration per batch
        for inputs, labels in dataloaders['test']:
            inputs, labels = inputs.to(device), labels.to(device)
            logps = model.forward(inputs)
            batch_loss = criterion(logps, labels)

            test_loss += batch_loss.item()

            # Calculate accuracy
            ps = torch.exp(logps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            qty_processed_inputs += len(equals)
            accuracy_sum += torch.sum(equals.type(torch.FloatTensor)).item()

    print(f"Test loss: {test_loss/len(dataloaders['test']):.3f}.. "
          f"Test accuracy: {accuracy_sum/qty_processed_inputs:.3f}")
    model.train()

print('Test complete!')

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint 
# Assumes training is complete and model is ready for inference
checkpoint = {'classifier': classifier_struct,
              'class_to_idx': image_datasets['training'].class_to_idx,
              'model_state_dict': model.state_dict(),
             }

In [ ]:
torch.save(checkpoint, 'checkpoint.pth')

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    
    model.class_to_idx = checkpoint['class_to_idx']
    model.classifier = nn.Sequential(checkpoint['classifier'])
    model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
load_checkpoint('checkpoint.pth')

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
from PIL import Image

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model
    
    # Resize and crop
    width, height = image.size
    old_short, old_long = min([width, height]), max([width, height])
    new_short = 256
    new_long = int(new_short*old_long/old_short)
    new_size = [new_short, new_long] if old_short == width else [new_long, new_short]
    image = image.resize(new_size)
    
    center_size = 224
    width, height = image.size
    left = (width - center_size)/2
    right = (width + center_size)/2
    top = (height - center_size)/2
    bottom = (height + center_size)/2
    image = image.crop((left, top, right, bottom))
    width, height = image.size
    
    # Normalize
    np_image = np.array(image)/255.0
    np_image -= [0.485, 0.456, 0.406]
    np_image /= [0.229, 0.224, 0.225]
    
    # Transpose
    np_image = np_image.transpose(2, 0, 1)
    return np_image
    

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()

    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    if title is not None:
        ax.set_title(title)
    
    return ax

In [ ]:
# Test process_image function

#im = Image.open('flowers/train/102/image_08007.jpg')
im = Image.open('flowers/test/82/image_01609.jpg')
#im = Image.open('flowers/test/82/image_01686.jpg')
new_im = process_image(im)
imshow(torch.tensor(new_im))


## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    # Retrieve file and covert to tensor
    f = Image.open(image_path)
    input_im = torch.tensor(process_image(f)).type(torch.FloatTensor)
    
    # Retrieve label from input image path
    label_class = Path(image_path).parts[2]
    
    # Switch class mapping for convenience
    idx_to_class = {d:k for k,d in model.class_to_idx.items()}
    
    model.eval()
    
    with active_session():
        with torch.no_grad():
            input_im = input_im.to(device)
            logps = model(input_im.unsqueeze_(0))
            ps = torch.exp(logps)
            
            # Get top probabilities, indices
            top_p, top_idx = ps.topk(topk, dim=1)
            
            # Convert top indices to classes
            top_class = [idx_to_class[k] for k in top_idx.tolist()[0]]
            
    model.train()    
    input_im = input_im.squeeze().cpu()
    
    # Return top probabilities, corresponding classes, input image tensor, and correct label class
    return top_p.tolist()[0], top_class, input_im, label_class

In [ ]:
# Test predict function
probs, classes, img, label_class = predict('flowers/test/8/image_03291.jpg', model)
print(f'For flower of class {label_class}')
print(probs)
print(classes)
print()

probs, classes, img, label_class = predict('flowers/test/52/image_04200.jpg', model)
print(f'For flower of class {label_class}')
print(probs)
print(classes)
print()

probs, classes, img, label_class = predict('flowers/test/82/image_01609.jpg', model)
print(f'For flower of class {label_class}')
print(probs)
print(classes)
print()

probs, classes, img, label_class = predict('flowers/test/82/image_01686.jpg', model)
print(f'For flower of class {label_class}')
print(probs)
print(classes)
print()


## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes
def show_img_p(ps, classes, im, label_class):
    ''' Function for viewing an image and it's predicted classes.
    '''
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10,10))
    ax1 = imshow(im, ax=ax1, title=cat_to_name[label_class])
    ax1.tick_params(axis='x', bottom=False, labelbottom=False)
    ax1.tick_params(axis='y', left=False, labelleft=False)
    ax2.barh(np.arange(len(ps)), ps)
    ax2.set_aspect(0.2)
    ax2.set_yticks(np.arange(len(classes)))

    ax2.set_yticklabels([cat_to_name[k] for k in classes], size='medium');
    ax2.set_title('Class Probability')
    ax2.set_xlim(0, max(ps)+0.1)

    plt.tight_layout()

In [ ]:
# Test show_img_p function
#probs, classes, img, label_class = predict('flowers/test/8/image_03291.jpg', model)
#probs, classes, img, label_class = predict('flowers/test/52/image_04200.jpg', model)
#probs, classes, img, label_class = predict('flowers/test/82/image_01609.jpg', model)
probs, classes, img, label_class = predict('flowers/test/82/image_01686.jpg', model)
show_img_p(probs, classes, img, label_class)